# Chapter 14: Recurrent Neural Networks

## Setup

In [1]:
import tensorflow as tf

In [27]:
# Cool Way to Visualize the Graph inside of Jupyter Notebook
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))


In [53]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import os

# Plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import pickle
import copy

In [54]:
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

In [55]:
def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [56]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [57]:
reset_graph()

### Prep Data

In [58]:
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        data = f.read()

    return data

In [59]:
source_path = './data/small_vocab_en'
target_path = './data/small_vocab_fr'

source_text = load_data(source_path)
target_text = load_data(target_path)

english_sentences = source_text.split('\n')
french_sentences = target_text.split('\n')

In [60]:
len(english_sentences)

137861

In [61]:
len(french_sentences)

137861

In [83]:
smol_english = english_sentences[:50]
smol_french = french_sentences[:50]

x_train = np.zeros(shape=(50, None), dtype=object)
y_pred = np.zeros(shape=(50, None), dtype=object)
# x_train = [0] * 50
# y_pred = [0] * 50

for i in range(0, len(x_train)):
    eng_words = smol_english[i].split(' ')
    fre_words = smol_french[i].split(' ')
    
    for j in range(0, len(smol_english[i].split(' '))):
         x_train[i][j] = eng_words[j]
    
    for j in range(0, len(smol_english[i].split(' '))):
         y_pred[i][j] = fre_words[j]
        
#     x_train[i] = np.array(smol_english[i].split(' '))
#     y_pred[i] = np.array(smol_french[i].split(' '))
#     x_train[i] = np.array(smol_english[i].split(' '))
#     y_pred[i] = np.array(smol_french[i].split(' '))
    
#     x_train = np.array(x_train)
#     y_pred = np.array(y_pred)
    
x_train.shape

TypeError: 'NoneType' object cannot be interpreted as an integer

### Building a Simple Encoder-Decoder Network

In [63]:
n_steps = 50
n_neurons = 200
n_layers = 3
num_encoder_symbols = 20000
num_decoder_symbols = 20000
embedding_size = 150
learning_rate = 0.01

In [64]:
X = tf.placeholder(tf.int32, [None, n_steps]) # English sentences
Y = tf.placeholder(tf.int32, [None, n_steps]) # French translations
W = tf.placeholder(tf.float32, [None, n_steps - 1, 1])
Y_input = Y[:, :-1]
Y_target = Y[:, 1:]

**X:**  Tensor("Placeholder:0", shape=(?, 50), dtype=int32)

**Y:**  Tensor("Placeholder_1:0", shape=(?, 50), dtype=int32)

**W:**  Tensor("Placeholder_2:0", shape=(?, 49, 1), dtype=float32)

**Y_input:**  Tensor("strided_slice:0", shape=(?, 49), dtype=int32)

**Y_target:**  Tensor("strided_slice_1:0", shape=(?, 49), dtype=int32)

In [65]:
print('X: ', X)
print('Y: ', Y)
print('W: ', W)
print('Y_input: ', Y_input)
print('Y_target: ', Y_target)

X:  Tensor("Placeholder:0", shape=(?, 50), dtype=int32)
Y:  Tensor("Placeholder_1:0", shape=(?, 50), dtype=int32)
W:  Tensor("Placeholder_2:0", shape=(?, 49, 1), dtype=float32)
Y_input:  Tensor("strided_slice:0", shape=(?, 49), dtype=int32)
Y_target:  Tensor("strided_slice_1:0", shape=(?, 49), dtype=int32)


In [66]:
encoder_inputs = tf.unstack(tf.transpose(X)) # list of 1D tensors
decoder_inputs = tf.unstack(tf.transpose(Y_input)) # list of 1D tensors

lstm_cells = [tf.nn.rnn_cell.BasicLSTMCell(num_units=n_neurons)
              for layer in range(n_layers)]
cell = tf.nn.rnn_cell.MultiRNNCell(lstm_cells)

output_seqs, states = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(
    encoder_inputs,
    decoder_inputs,
    cell,
    num_encoder_symbols,
    num_decoder_symbols,
    embedding_size)

logits = tf.transpose(tf.unstack(output_seqs), perm=[1, 0, 2])

In [67]:
logits_flat = tf.reshape(logits, [-1, num_decoder_symbols])
Y_target_flat = tf.reshape(Y_target, [-1])
W_flat = tf.reshape(W, [-1])
xentropy = W_flat * tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y_target_flat, logits=logits_flat)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

### Use the Model

In [78]:
with tf.Session() as sess:
    init.run()
    
    sess.run(y_pred, feed_dict={X: x_train})
    
#     for step in range(n_steps):
#         print("\rIteration: {}".format(step), end="\t")
#         feed_dict = {}
#     y_pred = 

ValueError: setting an array element with a sequence.

In [ ]:
show_graph(tf.get_default_graph())